In [ ]:
import os
import numpy as np
import glob
import shutil

import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
#import packages
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = 

base_dir = 

In [ ]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [ ]:
for cl in classes:
  img_path = 
  images = 
  print("{}: {} Images".format(cl, len(images)))
  train, val = 

  for t in train:
    

  for v in val:
      

In [ ]:
train_dir = 
val_dir = 

In [ ]:
print(len(train_dir))

In [ ]:
batch_size = 
IMG_SHAPE = 

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    

In [ ]:
image_gen_train = 


train_data_gen = 

In [ ]:
augmented_images = 
plotImages(augmented_images)

In [ ]:
image_gen_val = 
val_data_gen = 

In [ ]:
model = Sequential()


In [ ]:
# Compile the model
model.compile( )

In [ ]:
model.summary()

In [ ]:
EPOCHS = 
TRAIN_STEPS, TEST_STEPS = 
history = model.fit_generator( )

In [ ]:
acc = 
val_acc = 

loss = 
val_loss = 

epochs_range = 

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)


plt.subplot(1,2,2)

